In [ ]:
#importing the required libraries
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
cifar_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
data = cifar_trainset.data / 255 # data is numpy array

clear_output()

mean = data.mean(axis = (0,1,2)) 
std = data.std(axis = (0,1,2))
print(f"Mean : {mean}   STD: {std}")

Mean : [0.49139968 0.48215841 0.44653091]   STD: [0.24703223 0.24348513 0.26158784]


In [ ]:
#loading the CIFAR10 dataset

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

batch_size = 64

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_set = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_set = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

clear_output()

In [ ]:
#creating the VGG19 model

VGG_19 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']

class VGG19_net(nn.Module):
  def __init__(self, in_channels = 3, no_of_classes = 10):
    super(VGG19_net, self).__init__()
    self.in_channels = in_channels
    self.no_of_classes = no_of_classes
    self.conv_layers = self.create_conv_layers(VGG_19)
    self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    self.fully_connected_layers = nn.Sequential(
                                                nn.Linear(in_features = 512 * 1 * 1, out_features = 4096, bias = True),
                                                nn.ReLU(inplace = True),
                                                nn.Dropout(p = 0.5),
                                                nn.Linear(in_features = 4096, out_features = 4096, bias = True),
                                                nn.ReLU(inplace = True),
                                                nn.Dropout(p = 0.5),
                                                nn.Linear(in_features = 4096, out_features = self.no_of_classes),
                                              )

  def create_conv_layers(self, arch):
   layers = []
   in_channels = self.in_channels
   for layer in arch:
     if(type(layer) == int):
       out_channels = layer

       layers += [nn.Conv2d(
                            in_channels = in_channels,
                            out_channels = out_channels,
                            kernel_size = 3,
                            stride = 1,
                            padding = 1
                          ),
                  nn.BatchNorm2d(layer),
                  nn.ReLU(inplace = True)
                 ]
       in_channels = layer
     elif(layer == 'M'):
       layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)]

   return nn.Sequential(*layers)    

  def forward(self, x):
    x = self.conv_layers(x)
    x = self.avg_pool(x)
    x = torch.flatten(x, start_dim = 1, end_dim = -1)
    x = self.fully_connected_layers(x)
    return x

In [ ]:
#passing data and training the model

device = "cuda" if torch.cuda.is_available() else "cpu"

learning_rate = 3e-4
no_of_epochs = 10

vgg = VGG19_net().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg.parameters(), lr = learning_rate)

for epoch in range(no_of_epochs):
  for image, label in train_set:
    image = image.to(device)
    label = label.to(device)
    output = vgg(image)
    loss = loss_function(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print("Epoch", epoch + 1, "completed!")

print("Training of VGG-19 neural network successfully completed!")

Epoch 1 completed!
Epoch 2 completed!
Epoch 3 completed!
Epoch 4 completed!
Epoch 5 completed!
Epoch 6 completed!
Epoch 7 completed!
Epoch 8 completed!
Epoch 9 completed!
Epoch 10 completed!
Training of VGG-19 neural network successfully completed!


In [ ]:
#what does the output look like

with torch.no_grad():
  for image, label in test_set:
    image = image.to(device)
    label = label.to(device)
    outputs = vgg(image)
    temp, pred = torch.max(outputs, dim = 1)
    print(temp)
    print(pred)
    print("\n")
    print(temp[0])
    print(pred[0])
    print(label[0])
    break

tensor([ 8.1878,  4.1189,  2.5021,  1.7105,  3.6367,  3.6147,  6.2594,  2.9525,
         3.1035,  2.4162,  1.0373,  1.4131,  1.3374, 13.9563,  8.4483,  5.9720,
         2.0410,  3.9827,  4.4832, 11.8724,  3.2713,  5.2645,  3.4421,  6.2401,
         2.9163,  3.9559,  7.4412,  3.8710,  3.0241,  3.8694,  4.5801,  6.0913,
        15.1977, 11.5616,  4.0502,  1.5650,  3.9265,  3.8631,  5.0691,  6.0464,
         6.3270,  4.3630,  4.7962,  1.5815,  9.5685,  4.7860,  9.2943,  3.3330,
        11.1020,  5.0511,  1.2550,  3.1782,  3.1576,  8.3472,  8.0436,  6.0602,
         5.1455,  6.0653,  1.6465,  7.1832,  7.9633,  9.3108,  5.2932,  5.9856],
       device='cuda:0')
tensor([7, 7, 6, 3, 0, 9, 9, 5, 9, 0, 2, 4, 3, 1, 9, 5, 3, 6, 7, 8, 4, 9, 5, 7,
        0, 1, 2, 6, 5, 3, 7, 6, 8, 1, 4, 7, 6, 0, 4, 6, 4, 0, 6, 5, 2, 5, 1, 3,
        1, 5, 3, 1, 3, 1, 1, 5, 4, 9, 4, 5, 9, 1, 9, 4], device='cuda:0')


tensor(8.1878, device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')


In [ ]:
#testing the model

correct = 0
incorrect = 0

with torch.no_grad():
  for image, label in test_set:
    image = image.to(device)
    label = label.to(device)
    outputs = vgg(image)
    temp, pred = torch.max(outputs, dim = 1)
    correct += (pred == label).sum().item()
    incorrect += (pred != label).sum().item()

model_accuracy = (correct/(correct + incorrect)) * 100
print("Accuracy of the VGG-19 model : ", model_accuracy, "%", sep = "")

Accuracy of the VGG-19 model : 82.25%
